The first step in applying the Zel'dovich Control Variates (ZCV) is running the "preparation steps".

1. Save the window function (given k-binning) and the Zel'dovich theoretical prediction (given k-binning and redshift):

`python -m abacusnbody.hod.zcv.zenbu_window --path2config $1 --alt_simname $2`

2. Save the initial conditions fields (delta, delta^2, s^2, nabla^2):

`python -m abacusnbody.hod.zcv.ic_fields --path2config $1 --alt_simname $2`

3. Save the advected fields and their power spectra at this redshift

    - When applying ZCV to the power spectrum multipoles:

        - Run this version if planning to use mocks with and without RSD effects:

        `python -m abacusnbody.hod.zcv.advect_fields --path2config $1 --want_rsd --alt_simname $2`

        - Run this version if not planning to use mocks with RSD effects:

        `python -m abacusnbody.hod.zcv.advect_fields --path2config $1 --alt_simname $2`

    - When applying ZCV to the correlation function multipoles:

        - Run this version if planning to use mocks with and without RSD effects:

        `python -m abacusnbody.hod.zcv.advect_fields --path2config $1 --want_rsd --alt_simname $2 --save_3D_power`

        - Run this version if not planning to use mocks with RSD effects:

        `python -m abacusnbody.hod.zcv.advect_fields --path2config $1 --alt_simname $2 --save_3D_power`
        
An example `yaml` configuration file can be found in the `config/` directory, `lrg_hod_base_z0.500_nmesh576.yaml`. Note that the `--alt_simname` argument is optional and only needed if the user wants to use a different simulation from the one specified in the `yaml` file.


In [8]:
# Load necessary packages
import os

import numpy as np
import matplotlib.pyplot as plt
import yaml

from abacusnbody.hod.abacus_hod import AbacusHOD


ModuleNotFoundError: No module named 'yaml'

In [2]:
# Whether to use the last computed tracer power spectra (saves time)
load_presaved = False

In [3]:
# Load the config file and parse in relevant parameters
path2config = "config/lrg_hod_base_z0.500_nmesh576.yaml"

# Read the parameters from the yaml file
config = yaml.safe_load(open(path2config))
sim_params = config['sim_params']
HOD_params = config['HOD_params']
clustering_params = config['clustering_params']
zcv_params = config['zcv_params']

# Additional parameter choices
want_rsd = HOD_params['want_rsd']
write_to_disk = HOD_params['write_to_disk']
z_mock = sim_params['z_mock']
sim_name = sim_params['sim_name']
nmesh = zcv_params['nmesh']

NameError: name 'yaml' is not defined

In [4]:
# Run hod
newBall = AbacusHOD(sim_params, HOD_params, clustering_params)
mock_dict = newBall.run_hod(newBall.tracers, want_rsd, write_to_disk, Nthread=16)
nobj = mock_dict['LRG']['mass'].size
print("number of galaxies", nobj)

NameError: name 'AbacusHOD' is not defined

In [5]:
# Run zcv on the correlation function multipoles
zcv_dict = newBall.apply_zcv_xi(mock_dict, config, load_presaved=load_presaved)
print(zcv_dict.keys())
for key in zcv_dict.keys():
    if "ell" in key:
        print(key, zcv_dict[key])
        print("-----------------")

NameError: name 'newBall' is not defined

In [6]:
# Run zcv on the power spectrum multipoles
zcv_dict = newBall.apply_zcv(mock_dict, config, load_presaved=load_presaved)
print(zcv_dict.keys())
for key in zcv_dict.keys():
    if "ell" in key:
        print(key, zcv_dict[key])
        print("-----------------")

NameError: name 'newBall' is not defined

In [9]:
# Parse the output from the ZCV-reduced power spectrum measurements
k_binc = zcv_dict['k_binc']
pk_nn_betasmooth = zcv_dict['Pk_tr_tr_ell_zcv']
pk_tt_poles = zcv_dict['Pk_tr_tr_ell']
pk_zz = zcv_dict['Pk_ZD_ZD_ell']
pk_zn = zcv_dict['Pk_tr_ZD_ell']
r_zt = zcv_dict['rho_tr_ZD']
pk_zenbu = zcv_dict['Pk_ZD_ZD_ell_ZeNBu']

NameError: name 'zcv_dict' is not defined

Plot the ZCV reduction on the power spectrum.

In [10]:
# Set up the canvas
if want_rsd:
    figsize = (12, 5)
    n_ell = 3
else:
    figsize = (6, 5)
    n_ell = 1
f, ax = plt.subplots(1, n_ell, sharex=True, sharey=True, figsize=figsize)
if not want_rsd:
    ax = [ax]
    
# Loop over all multipoles
for ell in range(n_ell):
    
    if want_rsd:
        pk_zz_ell = pk_zz[ell, :].flatten()
        pk_zenbu_ell = pk_zenbu[ell, :].flatten()
        pk_tt_poles_ell = pk_tt_poles[0, ell, :]
        pk_nn_betasmooth_ell = pk_nn_betasmooth[ell, :]
    else:
        pk_zz_ell = pk_zz.flatten()
        pk_zenbu_ell = pk_zenbu.flatten()
        pk_tt_poles_ell = pk_tt_poles.flatten()
        pk_nn_betasmooth_ell = pk_nn_betasmooth.flatten()
        
    ax[ell].plot(k_binc, k_binc * pk_tt_poles_ell, label='Measured tracer power')
    ax[ell].plot(k_binc, k_binc * pk_nn_betasmooth_ell, label='Reduced tracer power')
    ax[ell].plot(k_binc, k_binc * pk_zenbu_ell, label='ZeNBu fitted power')

    if ell == 0:
        ax[ell].set_ylabel(r"$k P(k)$")
    ax[ell].set_xlabel(r"$k \ [h/{\rm Mpc}]$")
plt.legend()


NameError: name 'want_rsd' is not defined